In [1]:
import pandas as pd
from datetime import date
from utilities.logger import StandardLogger
from utilities.yahoo_data import YahooDataLoader
from model.grandma_valuation import GrandmaRegression

logger = StandardLogger('../_log/')
logPrint = logger.logPrint


In [2]:
l_instrument = {
    'SP500':'IVV',
    'Greater China':'3073.HK',
    'Developed Asia': 'VPL',
    'Europe':'IEV',
    'SE Asia': 'ASEA',

    'Global Healthcare': 'IXJ',
    'Global Consumer Staple': 'KXI',
    'US Utility': 'XLU'
}

l_metrics = [] # to store the valuation metrics

for name, ticker in l_instrument.items():
    logPrint(f"{name}: {ticker}")

    # Refresh data
    yahoo = YahooDataLoader(ticker, printfunc=logPrint)
    df = yahoo.queryEOD(save=True)

    # Fit model
    grandma = GrandmaRegression(recent_months=0, train_years=10)
    df_train, df_recent = grandma.fitTransform(df, col_price='close_adj')
    d_metrics = grandma.evaluateValuation()
    df_metrics = pd.Series(d_metrics).to_frame().T
    df_metrics['ticker'] = ticker
    df_metrics['name'] = name
    l_metrics.append(df_metrics)

    # Plot image
    fig = grandma.plotTrendline(title=name)
    fig.write_image(f"output/images/{ticker}.jpeg")

    logPrint('')


SP500: IVV
IVV: Existing EOD data file found at data/IVV_EOD.csv.gz.
IVV: Existing EOD data file contains 5464 rows over 5464 dates from 2000-05-19 to 2022-02-04.
IVV: Queried EOD data contains 1 rows over 1 dates from 2022-02-04 to 2022-02-04.
IVV: Amended data file contains 5464 rows over 5464 dates from 2000-05-19 to 2022-02-04.
Train data contains 2518 rows over 2518 dates from 2012-02-06 to 2022-02-04.
No recent data specified.
Fit regression... 244 out of 2518 dates are outliers... Re-fit wihtout outliers... No recent data to estimate... done!
Train RMSE = 13.4. Annualized Return = 0.134 over 10.0 years. Compared to base price 4.07e+02, the current price 4.51e+02 is over-valued by 0.107 or 0.796 years.

Greater China: 3073.HK
3073.HK: Existing EOD data file found at data/3073.HK_EOD.csv.gz.
3073.HK: Existing EOD data file contains 2808 rows over 2808 dates from 2010-09-15 to 2022-02-04.
3073.HK: Queried EOD data contains 1 rows over 1 dates from 2022-02-04 to 2022-02-04.
3073.HK:

In [3]:
# Consolidate valuation metrics
df_metrics = pd.concat(l_metrics).reset_index(drop=True)

cols_first = ['ticker', 'name']
df_metrics = df_metrics[cols_first + list(df_metrics.columns.drop(cols_first))]
df_metrics.drop(columns=['rmse_train'], inplace=True)

df_metrics.to_csv(f'output/valuation_metrics_{date.today()}.csv', index=False)

df_metrics


,ticker,name,train_years,annualized_return,currenct_price,base_price,over_value_range,over_value_years
0,IVV,SP500,10.00274,0.133939,450.850006,407.404486,0.106640,0.796184
1,3073.HK,Greater China,10.00274,0.082656,50.939999,52.267422,-0.025397,-0.002099
2,VPL,Developed Asia,10.00274,0.067575,74.699997,76.973890,-0.029541,-0.001996
3,IEV,Europe,10.00274,0.052666,52.860001,49.024203,0.078243,1.485648
4,ASEA,SE Asia,10.00274,0.016214,15.350000,14.314579,0.072333,4.461031
5,IXJ,Global Healthcare,10.00274,0.111017,84.190002,82.487695,0.020637,0.185892
6,KXI,Global Consumer Staple,10.00274,0.071560,62.869999,60.707307,0.035625,0.497834
7,XLU,US Utility,10.00274,0.107046,68.480003,69.371026,-0.012844,-0.001375
